# NS‑3 Code Generator with **Gemini 2.5 Flash**
This notebook batch‑generates ns‑3 C++ simulations from natural‑language descriptions.
It reads an *instruction prompt* from `prompt.yaml`, iterates over every `.txt` file found in `Dataset/Prompts/Large` and `Dataset/Prompts/Small`, and writes the generated code to `BasicEvaluation/Gemini/<name>.cc`.

Run the cells top‑to‑bottom.

## ⚙️ Install Python dependencies

In [1]:
%pip install -q -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


## 📦 Imports & logging setup

In [2]:
from pathlib import Path
import os, re, yaml, logging
from dotenv import load_dotenv
import google.generativeai as genai
from rich.logging import RichHandler
from tqdm import tqdm
import time

logging.basicConfig(level='INFO', handlers=[RichHandler()], format='%(message)s')

## 🛠 Configuration

In [3]:
PROMPT_YAML = Path('../prompt.yaml')
SRC_DIRS = [Path('../Dataset/Prompts/Large'), Path('../Dataset/Prompts/Small')]
OUT_DIR  = Path('Basic/Gemini')
MODEL_NAME = 'gemini-2.5-flash-preview-05-20'
CODE_BLOCK_RE = re.compile(r"```(?:cpp|c\+\+)?\s*([\s\S]+?)```", re.IGNORECASE)

### Helper: extract first fenced C++ block

In [4]:
def extract_code(text: str) -> str:
    m = CODE_BLOCK_RE.search(text)
    return m.group(1).strip() if m else text.strip()

## 📑 Load instruction prompt

In [5]:
instruction_prompt = yaml.safe_load(PROMPT_YAML.open())['instruction_prompt'].strip()
print(instruction_prompt)

You are a computer networking expert that works with network simulation in ns-3.
Your task is to generate **only** the ns-3 C++ source code for the given program description.
Output a single compilable .cc file – no commentary, no markdown fences, no explanations.
Use ns-3 public APIs, assume ns-3 ≥ 3.41, and write error-free C++17.


## 🔑 Authenticate & initialise Gemini

In [6]:
load_dotenv()
assert 'GEMINI_API_KEY' in os.environ, 'Add GEMINI_API_KEY to your .env file'
genai.configure(api_key=os.environ['GEMINI_API_KEY'])
model = genai.GenerativeModel(MODEL_NAME)
print('Model ready!')

Model ready!


## 🗂 Collect description files

In [7]:
txt_files = []
for d in SRC_DIRS:
    txt_files.extend(sorted(d.glob('*.txt')))
print(f'Found {len(txt_files)} description files')
OUT_DIR.mkdir(parents=True, exist_ok=True)

Found 400 description files


## 🚀 Generate ns‑3 code

In [8]:
for src in tqdm(txt_files, desc='Generating'):
    description = src.read_text(encoding='utf-8').strip()
    prompt = f"{instruction_prompt}\n\nProgram description:\n\"\"\"\n{description}\n\"\"\""
    try:
        resp = model.generate_content(prompt)
        code = extract_code(resp.text)
        outfile = OUT_DIR / f"{src.stem}.cc"
        outfile.write_text(code, encoding='utf-8')
        logging.info(f'✓ {src.name} → {outfile}')
    except Exception as e:
        logging.error(f'✗ {src.name} – {e}')

    time.sleep(7)

Generating:   0%|          | 0/400 [00:00<?, ?it/s]

[05/22/25 19:27:38] INFO     ✓ 1.txt → Basic\Gemini\1.cc                                            ]8;id=63295;file://C:\Users\Tasnim\AppData\Local\Temp\ipykernel_16732\1564753407.py\1564753407.py]8;;\:]8;id=442635;file://C:\Users\Tasnim\AppData\Local\Temp\ipykernel_16732\1564753407.py#9\9]8;;\

Generating:   0%|          | 1/400 [00:19<2:11:43, 19.81s/it]

[05/22/25 19:28:15] INFO     ✓ 10.txt → Basic\Gemini\10.cc                                          ]8;id=801435;file://C:\Users\Tasnim\AppData\Local\Temp\ipykernel_16732\1564753407.py\1564753407.py]8;;\:]8;id=827314;file://C:\Users\Tasnim\AppData\Local\Temp\ipykernel_16732\1564753407.py#9\9]8;;\

Generating:   0%|          | 2/400 [00:56<3:15:41, 29.50s/it]

...

Generating: 100%|█████████▉| 398/400 [4:08:36<00:56, 28.38s/it]

[05/22/25 23:36:13] INFO     ✓ 399.txt → Basic\Gemini\399.cc                                        ]8;id=703254;file://C:\Users\Tasnim\AppData\Local\Temp\ipykernel_16732\1564753407.py\1564753407.py]8;;\:]8;id=777851;file://C:\Users\Tasnim\AppData\Local\Temp\ipykernel_16732\1564753407.py#9\9]8;;\

Generating: 100%|█████████▉| 399/400 [4:08:54<00:25, 25.48s/it]

[05/22/25 23:36:38] INFO     ✓ 400.txt → Basic\Gemini\400.cc                                        ]8;id=408362;file://C:\Users\Tasnim\AppData\Local\Temp\ipykernel_16732\1564753407.py\1564753407.py]8;;\:]8;id=976837;file://C:\Users\Tasnim\AppData\Local\Temp\ipykernel_16732\1564753407.py#9\9]8;;\

Generating: 100%|██████████| 400/400 [4:09:19<00:00, 37.40s/it]
